In [ ]:
!pip install click requests tqdm pyspng ninja imageio-ffmpeg==0.4.3

In [ ]:
import torch
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from mpl_toolkits.axes_gridl import ImageGrid

from PIL import Image

import numpy as np

import copy
import pickle
import os

import matplotlib.pyplot as plt

from tqdm.notebook import tqdm

In [ ]:
%matplotlib inline

In [ ]:
# stylegan2-ada의 git을 복사하기
# align_images.py를 추가하기
if not os.path.isdir("stylegan2-ada-pytorch"):
  !git clone https://github.com/rkuo2000/stylegan2-ada-pytorch
%cd stylegan2-ada-pytorch

In [ ]:
if not os.path.isdir('pretrained'):
  !mkdir pretrained
  %cd pretrained
  !wget https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl
  %cd ..

학습된 모델 불러오기

In [ ]:
with open('pretrained/ffhq.pkl', 'rb') as f:
  G = pickle.load(f)['G_ema'].cuda()

샘플 만들기

In [ ]:
!gdown --id lzIEGC4uPVsjkZpL6be0UR2b4mQRq7rn9

In [ ]:
device_name = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device_name)

In [ ]:
w = torch.from_numpy(np.load('projected_w.npy')).cuda().unsqueeze(0)
img = G.synthesis(w, noise_mode='const')

In [ ]:
plt.imshow((img[0]*127.5 +128).permute(1,2,0).clamp(0, 255).detach().cpu().to(torch.uint8))
plt.axis('off')
plt.show()

In [ ]:
!gdown --id lwaSWjrS4StitDVtU2exsK9CWuk08kcX5
!unzip SVM_vectors.zip

In [ ]:
attribute_number = 39

In [ ]:
class_direction_Z = torch.load(f'SVM_vectors/class_direction_Z_on_3k_att{attribute_number}.pt').to(device)
class_direction_W = torch.load(f'SVM_vectors/class_direction_W_on_3k_att{attribute_number}.pt').to(device)

Z_intercept = torch.load(f'SVM_vectors/Z_intercept_on_3k_att{attribute_number}.pt').to(device)
W_intercept = torch.load(f'SVM_vectors/W_intercept_on_3k_att{attribute_number}.pt').to(device)

In [ ]:
def IFGAN_AllLayersEdition(tensors_for_edition, direction, class_direction, b, G, n_steps_var = 7, scale_factor = 1., save=False):
  """
  그림 편집을 latent space Z에서 InterFaceGAN 방식을 이용하여 편집하는 방법
  class 1, class 2 vector를 지정하고, 수직한 벡터로 반대로 이동하게 하여 특성을 집어넣음
  z: latent vectors
  direction: - class -1방향
  class_direction: data에서 hyperplane으로 수직으로 내린 벡터의 방향성
  b: 하이퍼 평면의 특성 관련된 것 - hyperplane의 위치 등
  G: Generator Network for StyleGAN2
  scale_factor: 변화의 정도를 지정하는 변수

  """
  rb_img = len(tensors_for_edition)

  steps = torch.linspace(0.0, 1.0, n_steps_var)* scale_factor
  fig = plt.figure(figsize=(2*n_steps_var, 2*nb_img))
  grid = ImageGrid(fig, 111, nrows_n_cols=(nb_img, n_steps_var), axes_pad=0.1)

  norm = torch.norm(class_direction)

  for i_img, tensor in enumerate(tensors_for_edition):
    for istep in range(n_steps_var):
      tensor_transformed = tensor.clone()

      distance = -(torch.dot(class_direction, tensor[0])+b)/norm**2 # tensor를 hyper plane으로 이동시키기 위함
      tensor_transformed += class_direction*distance + steps[istep]*class_direction*direction
      tensor_transformed = tensor_transformed.to(device)

      img = G.synthesis(tensor_transformed[None, :, :]).cpu().detach().numpy()[0].transpose((1,2,0))
      img = (img +1)/2
      iplot = i_img * n_steps_var + istep
      if istep == 0:
        grid[iplot].set_ylabel(f'$\mathrm{{IFG}}(\mathbf{{v}}_{{ att = {attribute_number}}}, \mathrm{{all}})$', rotation=0, size= 'x-large')
        grid[iplot].yaxis.set_label_coords(-0.5, 0.45)

      grid[iplot].imshow(np.clip((255*img).astype(int), 0, 255))
      grid[iplot].set_yticklabels([])
      grid[iplot].set_xticklabels([])
      grid[iplot].set_xticks([])
      grid[iplot].set_yticks([])
  plt.title = (f"{scale_factor=}")
  plt.legend()
  plt.show()
  if save:
    fig.savefig("all_components.png")

In [ ]:
IFGAN_AllLayersEdition(w, 1, class_direction_W, W_intercept, G, scale_factor=2.)

In [ ]:
class_direction_W = class_direction_W.to(device)
class_direction_Z = class_direction_Z.to(device)

W_intercept = W_intercept.to(device)
Z_intercept = Z_intercept.to(device)


In [ ]:
scale_factor = 1.
n_steps_var = 7
direction = 1

In [ ]:
ATTR_numbers = [31, 39, 15, 20, 22]
ATTRS = ['Smiling', 'Young', 'Eyeglasses', 'Male', 'Mustache']
directions = {}
intercepts = {}
for i, attr_name in enumerate(ATTRS):
  directions[attr_name] = torch.load(f'SVM_vectors/direction_W_on_3k_att{ATTR_numbers[i]}.pt').to(device)
  intercepts[attr_name] = torch.load(f'SVM_vectors/W_intercept_on_3k_att{ATTR_numbers[i]}.pt').to(device)


In [ ]:
#@title {display-mode: "form", run: "auto"}

Young =-2.4 #@param{type:"slider", min:-3.0, max:3.0, step:0.1}
Eyeglasses = 0 #@param{type:"slider", min:-2.9, max:3.0, step:0.1}
Male = 1.6 #@param{type:"slider", min:-3.0, max:3.0, step:0.1}
Smiling = 0.9 #@param{type:"slider", min:-3.0, max:3.0, step:0.1}
Mustache = 0 #@param{type:"slider", min:-3.0, max:3.0, step:0.1}

direction=1
tensor_transformed = w[0].clone()

for i, attr_name in enumerate(ATTRS):
  norm = torch.norm(directions[attr_name])

  tensor_transformed += eval(attr_name)*directions[attr_name]*direction

tensor_transformed = tensor_transformed.to(device)

img = G.synthesis(tensor_transformed[None, :, :]).cpu().detach().numpy()[0].transpose((1,2,0))
img = (img+1)/2
plt.imshow(np.clip((255*img).astype(int), 0, 255))
plt.axis('off')

In [ ]:
eval(attr_name)